In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define file paths
ais_tracks_path = '../data/tracks_ais.csv'
radar_tracks_path = '../data/tracks_radar.csv'
radar_detections_path = '../data/detections_radar.csv'

ais_tracks = pd.read_csv(ais_tracks_path)
radar_tracks = pd.read_csv(radar_tracks_path)
radar_detections = pd.read_csv(radar_detections_path)

In [2]:
features_subset = ['min_speed', 'max_speed', 'avg_speed', 'curviness', 'heading_mean', 'heading_std', 
 'turning_mean', 'turning_std', 'duration_z', 'distance', 'distance_o']

In [53]:
#datetime conversion
radar_detections['datetime'] = pd.to_datetime(radar_detections['cdate'] + ' ' + radar_detections['ctime'])

In [54]:
import numpy as np

def distance_between_points(lat1, lon1, lat2, lon2, r = 6371):
    """
    Calculate the distance between two points on the Earth's surface
    using the Haversine formula.

    Args:
        lat1 (float): Latitude of the first point
        lon1 (float): Longitude of the first point
        lat2 (float): Latitude of the second point
        lon2 (float): Longitude of the second point
        r (float): Radius of the Earth in kilometers
    """
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return r * c

In [69]:
#testing the distance function
track_1 = radar_detections[radar_detections['id_track'] == 29941750].copy()
track_1[['id_track', 'assoc_id']]

,id_track,assoc_id
0,29941750,29941830
1,29941750,29941830
2,29941750,29941830
3,29941750,29941830
4,29941750,29941830
...,...,...
115,29941750,29941830
116,29941750,29941830
117,29941750,29941830
118,29941750,29941830


In [80]:
track_1.loc[:, 'longitude_prev'] = track_1 ['longitude'].shift(1)
track_1.loc[:, 'latitude_prev'] = track_1 ['latitude'].shift(1)
track_1.loc[:, 'distance_travelled'] = distance_between_points(track_1['latitude_prev'], track_1['longitude_prev'], track_1['latitude'], track_1['longitude'])
track_1.loc[:, 'time_prev'] = track_1['datetime'].shift(1)
track_1.loc[:, 'time_diff'] = (track_1['datetime'] - track_1['time_prev']).dt.total_seconds()
track_1.loc[:, 'instant_speed'] = track_1['distance_travelled'] / track_1['time_diff']

total_distance = track_1['distance_travelled'].sum()
total_time = (track_1['datetime'].iloc[-1] - track_1['datetime'].iloc[0]).total_seconds() # in seconds
avg_speed_kmps = total_distance / total_time
avg_speed = avg_speed_kmps * 1943.84449 #in knots
max_speed = track_1['instant_speed'].max() * 1943.84449
min_speed = track_1['instant_speed'].min() * 1943.84449
print(total_time)
print(avg_speed, max_speed, min_speed)

755.0
3.7696156959220666 10.989409736102735 0.07204854856538934


In [81]:
debug_columns = ['id_track', 'datetime', 'longitude', 'latitude', 'longitude_prev', 'latitude_prev', 'distance_travelled', 'time_prev', 'time_diff', 'instant_speed']
track_1[debug_columns]

,id_track,datetime,longitude,latitude,longitude_prev,latitude_prev,distance_travelled,time_prev,time_diff,instant_speed
0,29941750,2023-07-02 11:40:11,-117.302575,32.815120,NaN,NaN,NaN,NaT,NaN,NaN
1,29941750,2023-07-02 11:40:17,-117.302553,32.815148,-117.302575,32.815120,0.003731,2023-07-02 11:40:11,6.0,0.000622
2,29941750,2023-07-02 11:40:24,-117.302543,32.815150,-117.302553,32.815148,0.000961,2023-07-02 11:40:17,7.0,0.000137
3,29941750,2023-07-02 11:40:30,-117.302525,32.815115,-117.302543,32.815150,0.004240,2023-07-02 11:40:24,6.0,0.000707
4,29941750,2023-07-02 11:40:36,-117.302515,32.815090,-117.302525,32.815115,0.002933,2023-07-02 11:40:30,6.0,0.000489
...,...,...,...,...,...,...,...,...,...,...
115,29941750,2023-07-02 11:52:22,-117.313735,32.820325,-117.313532,32.820215,0.022571,2023-07-02 11:52:15,7.0,0.003224
116,29941750,2023-07-02 11:52:28,-117.313870,32.820398,-117.313735,32.820325,0.015001,2023-07-02 11:52:22,6.0,0.002500
117,29941750,2023-07-02 11:52:34,-117.314072,32.820508,-117.313870,32.820398,0.022492,2023-07-02 11:52:28,6.0,0.003749
118,29941750,2023-07-02 11:52:40,-117.314207,32.820582,-117.314072,32.820508,0.015061,2023-07-02 11:52:34,6.0,0.002510


In [66]:
ais_tracks[ais_tracks['assoc_id'] == 29941750]

,id_track,id_site,id_m2,source,duration,alarm,min_speed,max_speed,avg_speed,curviness,...,dest,eta_month,eta_day,eta_hour,eta_minute,type_m2,sdate,stime,ldate,ltime
371,29941830,10,338417665-0702-1143,ais,2602.0,0,2.6,27.4,16.764286,2.066154,...,NaN,NaN,NaN,NaN,NaN,class_b,2023-07-02,11:43:18,2023-07-02,12:26:40


15345
